In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 2    # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_exact_50-2.csv")

  1%|▍                                                                              | 6/1000 [02:03<5:41:39, 20.62s/it]


KeyboardInterrupt: 

## Heuristic - Round

In [ ]:
from src.heuristic import naive_round

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rnd_50-2.csv")

## Heuristic - RENS

In [ ]:
from src.heuristic import rens

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    # set params
    model.set_param_val({"p":p, "a":a})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = rens(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_rens_50-2.csv")

## Heuristic - N1

In [ ]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-2.csv")

## Heuristic - N3

In [ ]:
model_heur = model.first_solution_heuristic(nodes_limit=3)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n3_50-2.csv")

## Heuristic - N5

In [ ]:
model_heur = model.first_solution_heuristic(nodes_limit=5)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n5_50-2.csv")

## Learnable Rounding

In [ ]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [ ]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [ ]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-2.csv")

## Learnable Threshold

In [ ]:
# hyperparameters
penalty_weight = 50   # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate
batch_size = 64       # batch size

In [ ]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd])
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [ ]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.Adam(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup)
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_test,a_test))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-2.csv")